In [ ]:
import pandas as pd

In [2]:
dat = pd.read_csv('data/DataFiles/RegularSeasonDetailedResults.csv')

In [3]:
dat.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14


### Advanced Metrics

Calculate:     
1) Offensive Efficiency - Possession based   
2) Defensive Efficiency - Possession based   
3) Net Rating - Possession based   
4) Assist Ratios      
5) eFG    

Maybe look for Dean Olivers Four Factor Formula 

In [4]:
# pos = possessions
dat['wPos'] = dat.apply(lambda row: 0.96*(row.WFGA + row.WTO + 0.44*row.WFTA - row.WOR), axis=1)
dat['lPos'] = dat.apply(lambda row: 0.96*(row.LFGA + row.LTO + 0.44*row.LFTA - row.LOR), axis=1)

In [5]:
dat['WOffRtg'] = dat.apply(lambda row: 100 * (row.WScore / row.wPos), axis=1)
dat['LOffRtg'] = dat.apply(lambda row: 100 * (row.LScore / row.lPos), axis=1)
#Defensive efficiency (DefRtg) = 100 x (Opponent points / Opponent possessions)
dat['WDefRtg'] = dat.LOffRtg
dat['LDefRtg'] = dat.WOffRtg
#Net Rating = Off.Rtg - Def.Rtg
dat['WNetRtg'] = dat.apply(lambda row:(row.WOffRtg - row.WDefRtg), axis=1)
dat['LNetRtg'] = dat.apply(lambda row:(row.LOffRtg - row.LDefRtg), axis=1)

In [6]:
#Assist Ratio : Percentage of team possessions that end in assists
dat['WAstR'] = dat.apply(lambda row: 100 * row.WAst / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
dat['LAstR'] = dat.apply(lambda row: 100 * row.LAst / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
#Turnover Ratio: Number of turnovers of a team per 100 possessions used.
#(TO * 100) / (FGA + (FTA * 0.44) + AST + TO)
dat['WTOR'] = dat.apply(lambda row: 100 * row.WTO / (row.WFGA + 0.44*row.WFTA + row.WAst + row.WTO), axis=1)
dat['LTOR'] = dat.apply(lambda row: 100 * row.LTO / (row.LFGA + 0.44*row.LFTA + row.LAst + row.LTO), axis=1)
                    
#eFG% : Effective Field Goal Percentage adjusting for the fact that 3pt shots are more valuable
dat['WeFGP'] = dat.apply(lambda row:(row.WFGM + 0.5 * row.WFGM3) / row.WFGA, axis=1)      
dat['LeFGP'] = dat.apply(lambda row:(row.LFGM + 0.5 * row.LFGM3) / row.LFGA, axis=1) 

In [7]:
#FTA Rate : How good a team is at drawing fouls.
dat['WFTAR'] = dat.apply(lambda row: row.WFTA / row.WFGA, axis=1)
dat['LFTAR'] = dat.apply(lambda row: row.LFTA / row.LFGA, axis=1)
                         
#OREB% : Percentage of team offensive rebounds
dat['WORP'] = dat.apply(lambda row: row.WOR / (row.WOR + row.LDR), axis=1)
dat['LORP'] = dat.apply(lambda row: row.LOR / (row.LOR + row.WDR), axis=1)
#DREB% : Percentage of team defensive rebounds
dat['WDRP'] = dat.apply(lambda row: row.WDR / (row.WDR + row.LOR), axis=1)
dat['LDRP'] = dat.apply(lambda row: row.LDR / (row.LDR + row.WOR), axis=1)                                      
#REB% : Percentage of team total rebounds
dat['WRP'] = dat.apply(lambda row: (row.WDR + row.WOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1)
dat['LRP'] = dat.apply(lambda row: (row.LDR + row.LOR) / (row.WDR + row.WOR + row.LDR + row.LOR), axis=1) 

In [8]:
dat.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WeFGP,LeFGP,WFTAR,LFTAR,WORP,LORP,WDRP,LDRP,WRP,LRP
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.491379,0.433962,0.310345,0.415094,0.388889,0.294118,0.705882,0.611111,0.542857,0.457143
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.483871,0.402985,0.306452,0.298507,0.375000,0.416667,0.583333,0.625000,0.488636,0.511364
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.482759,0.321918,0.500000,0.315068,0.435897,0.543860,0.456140,0.564103,0.447917,0.552083
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.513158,0.428571,0.815789,0.306122,0.230769,0.472222,0.527778,0.769231,0.403226,0.596774
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.540984,0.435484,0.213115,0.435484,0.531250,0.488372,0.511628,0.468750,0.520000,0.480000


In [9]:
dat['W4Factor'] = dat.apply(lambda row: .40*row.WeFGP + .25*row.WTOR + .20*row.WORP + .15*row.WFTAR, axis=1)
dat['L4Factor'] = dat.apply(lambda row: .40*row.LeFGP + .25*row.LTOR + .20*row.LORP + .15*row.LFTAR, axis=1)                                      

In [16]:
dat.head()

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,WFTAR,LFTAR,WORP,LORP,WDRP,LDRP,WRP,LRP,W4Factor,L4Factor
0,2003,10,1104,68,1328,62,N,0,27,58,...,0.310345,0.415094,0.388889,0.294118,0.705882,0.611111,0.542857,0.457143,5.962561,5.369097
1,2003,10,1272,70,1393,63,N,0,26,62,...,0.306452,0.298507,0.375000,0.416667,0.583333,0.625000,0.488636,0.511364,3.585450,3.453860
2,2003,11,1266,73,1437,61,N,0,24,58,...,0.500000,0.315068,0.435897,0.543860,0.456140,0.564103,0.447917,0.552083,2.965976,3.166090
3,2003,11,1296,56,1457,50,N,0,18,38,...,0.815789,0.306122,0.230769,0.472222,0.527778,0.769231,0.403226,0.596774,4.393078,5.993610
4,2003,11,1400,77,1208,71,N,0,30,61,...,0.213115,0.435484,0.531250,0.488372,0.511628,0.468750,0.520000,0.480000,4.129417,2.944616
